## Set up environment and fetch Files

In [28]:
#set up environment
import pandas as pd
import qiime2 as q2
import seaborn as sns
import matplotlib.pyplot as plt
import os

%matplotlib inline
# ! rm -r data ###unappend this line if you want to wipe clean the data folder
data_dir = 'data'
safe = 'permanent_dir'
! mkdir -p data
! mkdir -p permanent

In [26]:
# fetch files from polybox
! wget -r -np -nH --cut-dirs=3 -R "index.html*" "https://polybox.ethz.ch/index.php/s/56JaAiKdGwioBKN/download" -O polybox_files.tar.gz
! tar -xzf polybox_files.tar.gz

#unzip polybox files
!mv polybox_files.tar.gz polybox_files.zip
!unzip polybox_files.zip
!rm polybox_files.zip

#untar MAGs files and store everything in data_dir
!tar -xzf applied_bioinformatics/Illumina_MAGs.tar.gz -C ./data
!tar -xzf applied_bioinformatics/PacBio_MAGs.tar.gz -C ./data
!mv applied_bioinformatics/merged_metadata_filtered.tsv ./data
!rm -r applied_bioinformatics

will be placed in the single file you specified.

--2025-10-12 11:27:10--  https://polybox.ethz.ch/index.php/s/56JaAiKdGwioBKN/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘polybox_files.tar.gz’

polybox_files.tar.g     [       <=>          ] 492.28M   404MB/s    in 1.2s    

2025-10-12 11:27:11 (404 MB/s) - ‘polybox_files.tar.gz’ saved [516188455]

FINISHED --2025-10-12 11:27:12--
Total wall clock time: 2.2s
Downloaded: 1 files, 492M in 1.2s (404 MB/s)
gzip: stdin has more than one entry--rest ignored
tar: Child returned status 2
tar: Error is not recoverable: exiting now
Archive:  polybox_files.zip
   creating: applied_bioinformatics/
 extracting: applied_bioinformatics/.DS_Store  
 extracting: applied_bioinformatics/Illumina_MAGs.tar.gz  
 extracting: applied_bioinformatics/PacBio_M

## Create metadata df for exploration

In [34]:
from uuid import uuid4
# load metadata
metadata_df = pd.read_csv(f"{data_dir}/merged_metadata_filtered.tsv", sep="\t", index_col=0)

# rename all fasta files with UUIDs
for technique in os.listdir(data_dir):
    tech_path = os.path.join(data_dir, technique)
    if not os.path.isdir(tech_path):
        continue
    for sample_id in os.listdir(tech_path):
        sample_path = os.path.join(tech_path, sample_id)
        if not os.path.isdir(sample_path):
            continue
        for file in os.listdir(sample_path):
            if file.endswith((".fa", ".fasta")):
                old_path = os.path.join(sample_path, file)
                new_path = os.path.join(sample_path, f"{uuid4()}.fa")
                os.rename(old_path, new_path)

records = []

for technique in os.listdir(data_dir):
    tech_path = os.path.join(data_dir, technique)
    if not os.path.isdir(tech_path):
        continue
    for sample_id in os.listdir(tech_path):
        sample_path = os.path.join(tech_path, sample_id)
        if not os.path.isdir(sample_path):
            continue
        for f in os.listdir(sample_path):
            if f.endswith((".fa", ".fasta")):
                abs_path = os.path.abspath(os.path.join(sample_path, f))
                if sample_id in metadata_df.index:
                    mag_id = os.path.splitext(f)[0]  # filename without extension
                    records.append((sample_id, mag_id, abs_path))

# build dataframe
manifest_df = pd.DataFrame.from_records(records, columns=["sample-id", "mag-id", "filename"])

# save as MANIFEST (tab-separated, no index)
manifest_path = os.path.join(data_dir, "MANIFEST")
manifest_df.to_csv(manifest_path, sep="\t", index=False)

print(f"MANIFEST saved to: {manifest_path}")
print(manifest_df.head())
print(manifest_df.columns)

MANIFEST saved to: data/MANIFEST
  sample-id                                mag-id  \
0      M004  69e86027-4a0f-43c1-9398-068ab202a4cd   
1      M004  ace19be8-b8f7-41f7-b7d5-995892e5453d   
2      M004  9306eef0-7439-4f00-b0cc-f758a5be73e4   
3      M004  fa6cba61-7497-493a-8c6f-73096a22a507   
4      M004  8b5b4a73-4949-4ea1-873b-5e6ed751ddc1   

                                            filename  
0  /home/jovyan/Interplanetary_Microbiome/data/Pa...  
1  /home/jovyan/Interplanetary_Microbiome/data/Pa...  
2  /home/jovyan/Interplanetary_Microbiome/data/Pa...  
3  /home/jovyan/Interplanetary_Microbiome/data/Pa...  
4  /home/jovyan/Interplanetary_Microbiome/data/Pa...  
Index(['sample-id', 'mag-id', 'filename'], dtype='object')


## EXPLORATION

In [30]:
merged_df.columns

NameError: name 'merged_df' is not defined

In [51]:
merged_df['samp_country'].value_counts()

samp_country
Thailand       621
Laos           568
Benin          200
Slovenia        19
Germany         10
Switzerland      8
Name: count, dtype: int64

In [54]:
merged_df['category'].value_counts()

category
fermented fish          772
fermented meat          363
fermented legumes       127
fermented roots         120
fermented vegetables     33
fermented cereal         11
Name: count, dtype: int64

NameError: name 'MANIFEST' is not defined

In [2]:
!tar chvfz notebook.tar.gz *

Git_Master_Book.ipynb
README.md
data/
data/metadata_merged.csv
data/raw/
data/raw/PacBio_MAGs/
data/raw/PacBio_MAGs/M004/
data/raw/PacBio_MAGs/M004/.snakemake_timestamp
data/raw/PacBio_MAGs/M004/0a60b6c6-3bd9-481c-bfcc-51d57c7a0ca9.fa
data/raw/PacBio_MAGs/M004/e49c4f16-230d-4b7e-ba0f-375c80db800a.fa
data/raw/PacBio_MAGs/M004/eba701ac-0b74-4470-b00d-45613f5ab824.fa
data/raw/PacBio_MAGs/M004/f1b0abdd-50c9-4d1c-9f30-94836073f39e.fa
data/raw/PacBio_MAGs/M004/d4167ce6-b93d-4512-8b6e-0a459934d140.fa
data/raw/PacBio_MAGs/M004/be106cda-fef6-4651-9529-6cc70aa88e28.fa
data/raw/PacBio_MAGs/M004/52b7acde-3869-4f53-911b-a3932388f3fb.fa
data/raw/PacBio_MAGs/M004/bc79fe9e-6de8-4945-8484-70f89d94a814.fa
data/raw/PacBio_MAGs/M004/797f8dc8-88ca-4d7a-a95f-065a5d3af308.fa
data/raw/PacBio_MAGs/M004/1bba622a-08bc-41b3-b34c-5373d4de0a62.fa
data/raw/PacBio_MAGs/M004/2a91d0f7-b859-45fa-bca9-4f779d345bb5.fa
data/raw/PacBio_MAGs/M004/a23adcf6-a721-4134-93cf-9de112e1c7f4.fa
data/raw/PacBio_MAGs/M004/2c418eb6-745b